In [12]:
import numpy as np 
from PIL import Image 
from matplotlib import pyplot as plt

In [13]:
def CalculateColorMap(img):
    colorMap={}
    count=0
    width, height = img.size
    arr = [[0 for i in range(height)] for j in range(width)]
    for i in range (width):
        for j in range (height):
            r,g,b = img.getpixel((i,j))
            index = (r,g,b)
            if( not index in colorMap):
                colorMap[index]=count
                arr[i][j]=count
                count+=1
            else:
                arr[i][j]=colorMap[index]
    returnValue = {"colorMap":colorMap,"indexedImage":arr}            
    return returnValue 


In [14]:
def QuantizationLevels(colorMap,ourrange):
    removedColors = {}
    kept = {}
    for color in list(colorMap.items()):
        x = tuple(map(lambda i: i - ourrange, color[0]))
        y=tuple(map(lambda i: i +ourrange, color[0]))
        if(color[0] in colorMap ):
            kept[color[0]]=colorMap[color[0]]
            for color2 in list(colorMap.items()):
                all(x1 < y1 for x1, y1 in zip(color2[0], y))
                if(all(x1 > y1 for x1, y1 in zip(color2[0], x)) and  all(x1 < y1 for x1, y1 in zip(color2[0], y)) and (not color2[0] in kept)):
                    indexdel =colorMap[color2[0]]
                    indexkept = colorMap[color[0]]
                    removedColors[indexdel] = indexkept
                    del colorMap[color2[0]]
    return {"colorMap":colorMap,"removedColors":removedColors}
        
    
    

In [15]:
def AdjustIndex(arr,removedColors):
    for index1,col in enumerate(arr):
        for index2,entry in enumerate(col):
            if(entry in removedColors):
                arr[index1][index2] = removedColors[entry]
    return arr
    

In [16]:
def ReverseColorMap(colorMap):
    newColorMap={}
    for index in colorMap:
        value= colorMap[index]
        newColorMap[value]=index
    colorMap=newColorMap
    return newColorMap

In [17]:
def ColorMapToImage(arr,colorMap,imageName):
    img = Image.new('RGB', (len(arr), len(arr[0])), 'white')
    width = len(arr)
    height=len(arr[0])
    for i in range (width):
        for j in range (height):
            index= arr[i][j]
            img.putpixel((i,j),colorMap[index])
    img.save("output/"+imageName)

In [18]:
import os 
basedir = "input/"
our_range = 50
for image in os.listdir(basedir):
    img = Image.open(basedir+image)
    x= CalculateColorMap(img)
    colorMap = x["colorMap"]
    arr = x["indexedImage"]
    y = QuantizationLevels(colorMap,our_range)
    colorMap =  y["colorMap"]
    removedColors = y['removedColors']
    arr = AdjustIndex(arr,removedColors)
    colorMap=ReverseColorMap(colorMap)
    ColorMapToImage(arr,colorMap,image)